In [2]:
import numpy as np  # Librería para aplicar álgebra lineal
import pandas as pd # Para manejar los datos (datasets)

import matplotlib.pyplot as plt # Para las visualizaciones
import seaborn as sns # Visualizaciones más fancy

from sklearn import preprocessing # Para el procesador de los datos
from sklearn.preprocessing import Imputer # Para adoptar una estrategia para los missing values
from sklearn.preprocessing import LabelEncoder as Codificar # Para codificar variables categóricas
from sklearn.preprocessing import OneHotEncoder # Para pasar el LaberEncoder vector a OneHot matriz
from sklearn.preprocessing import MinMaxScaler # Para realizar el escalado en escala (0-1)
from sklearn.model_selection import train_test_split as Separar # Para dividir en los 2 conjuntos
#from statsmodels.tools.eval_measures import rmse # Para calcular el error
from sklearn.metrics import confusion_matrix as CM # Para construir la matriz de confusión
from matplotlib.colors import ListedColormap as Colors # Para pintar las regiones en Clasificación

In [3]:
#Se especifican los tipo de datos para mejorar la performance
events = pd.read_csv('/home/nicolas/Descargas/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv', dtype={"timestamp": object,#intenté hacerla datetime y no anduvo 
                                                                     "event": 'category',
                                                                     "person":object,
                                                                     "url":object,
                                                                     "sku":object,#intenté ponerle int y no anduvo, con float si, pero finalmente me di cuenta que no tiene sentido
                                                                     "model":object,
                                                                     "condition":'category',
                                                                     "storage":object,
                                                                     "color":'category',
                                                                     "skus":object,
                                                                     "search_term":object,
                                                                     "staticpage":object,
                                                                     "campaign_source":object,
                                                                     "search_engine":object,
                                                                     "channel":object,
                                                                     "new_vs_returning":'category',
                                                                     "city":object,
                                                                     "region":object,
                                                                     "country":object,
                                                                     "device_type":object,
                                                                     
                                                                    })
etiquetas = pd.read_csv('/home/nicolas/Descargas/fiuba-trocafone-tp2-final-set/labels_training_set.csv',dtype={"person":object, "label":bool})

/home/nicolas/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
(etiquetas['label'].value_counts())/etiquetas['label'].count()

False    0.949521
True     0.050479
Name: label, dtype: float64

In [5]:
events['timestamp'] =  pd.to_datetime(events['timestamp'])
events[['marca','modelo']] = events['model'].dropna().str.split(' ',n=1,expand=True)
events['weekday']=events['timestamp'].apply(lambda x: x.day_name())
events['hour']=events['timestamp'].dt.hour
events['month']=events['timestamp'].dt.month
events['day']=events['timestamp'].dt.day


In [30]:
events.groupby('person')['event'].value_counts().unstack()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0
00091926,15.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,372.0,34.0
00091a7a,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0
000ba417,1.0,24.0,6.0,1.0,14.0,NaN,1.0,NaN,NaN,153.0,6.0
000c79fe,1.0,NaN,1.0,NaN,1.0,NaN,1.0,9.0,NaN,3.0,1.0
000e4d9e,19.0,17.0,1.0,NaN,17.0,NaN,5.0,NaN,NaN,339.0,13.0
000e619d,6.0,11.0,1.0,NaN,8.0,NaN,3.0,6.0,NaN,28.0,5.0
001001be,NaN,NaN,3.0,1.0,3.0,NaN,NaN,17.0,NaN,41.0,3.0
0010e89a,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0


In [40]:
features= pd.DataFrame(events.groupby('person')['event'].value_counts().unstack().reset_index(),columns=['person','ad campaign hit','brand listing','checkout','conversion','generic listing','lead','search engine hit','searched products','staticpage','viewed product','visited site']).fillna(0)

In [41]:
features

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0,34.0
2,00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
3,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0
4,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0
5,000e4d9e,19.0,17.0,1.0,0.0,17.0,0.0,5.0,0.0,0.0,339.0,13.0
6,000e619d,6.0,11.0,1.0,0.0,8.0,0.0,3.0,6.0,0.0,28.0,5.0
7,001001be,0.0,0.0,3.0,1.0,3.0,0.0,0.0,17.0,0.0,41.0,3.0
8,0010e89a,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
9,0016c4b5,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
